In [1]:
%display latex

En esta notebook implementamos algunos programas que nos permitan calcular la curvatura de un espacio riemanniano homogéneo reductivo $M = G/H$. Vamos a usar las fórmulas de [Bes08]. **Hay que tener cuidado de no usar mal las identificaciones.** 

Supondremos que $\dim M = n$, $\dim H = m$, y $G \subset SO(N)$ es un grupo de Lie conexo, semisimple y compacto de dimensión $m + n$. Consideraremos en $G$ la métrica biinvariante definida por
$$\langle X, Y \rangle^{bi} = - \operatorname{trace} (XY), \qquad X, Y \in \mathfrak g$$

La geometría de $M$ queda determinada por la descomposición reductiva
$$\mathfrak g = \mathfrak h \oplus \mathfrak m$$
en donde $\mathfrak m$ es el complemento ortogonal de $\mathfrak h$ con respecto a la métrica biinvariante en $\mathfrak g$ junto con un producto interno $\operatorname{Ad}(H)$ invariante en $\mathfrak m$ el cual denotamos por
$\langle X, Y\rangle$ para $X, Y \in \mathfrak m$. Usamos el mismo símbolo $\langle -, -\rangle$ para denotar la métrica $G$ invariante en $\mathfrak m$. También estamos suponiendo que la acción de $G$ en $M$ es (casi) efectiva. 

**OBS.** Cada $X \in \mathfrak g$ induce un campo de Killing $X^*$ en $M$ definiendo
$$X^*_p = \frac{d}{dt}\Big|_0 \operatorname{Exp}(tX) \cdot p$$

Esta asignación cambia el signo del corchete de Lie: para todos $X, Y \in \mathfrak g$ se tiene
$$[X, Y]^* = -[X^*, Y^*]$$

Y si denotamos por $o = eH$ tenemos que la asignación $\mathfrak m \to T_oM$ dada por $X \in \mathfrak m \mapsto X^*_o$ es una isometría lineal (más aún es la diferencial de la proyección al cociente en la identidad).

**Implementación.**
* Para introducir $\mathfrak g$ daremos una lista con una base $\mathfrak g$
* Para introducir $\mathfrak h$ daremos una lista con una base de $\mathfrak h$
* Implementaremos programas que verifiquen las listas mencionadas en los ítems anteriores efectivamente den lugar a un álgebra de Lie y una subálgebra de Lie
* Para determinar $\mathfrak m$ construiremos una base ortonormal de $\mathfrak g$ cuyos primeros $m$ elementos formen una base ortonormal de $\mathfrak h$. Luego los últimos $n$ elementos serán una base de $\mathfrak m$
* El producto interno en $\mathfrak m$ deberá ser introducido explícitamente y habra que implementar un programa que chequee la Ad-invariancia.

In [2]:
def lie_bracket(X, Y):
    return X * Y - Y * X

def biinv_metric(X, Y):
    return - (X * Y).trace()

def is_li(gens):
    n = len(gens)
    M = matrix([gens[i].list() for i in range(n)])
    if M.rank() == n:
        return True
    else:
        return False

def is_lie_algebra(basis):
    if is_li(basis) == False:
        print("Not a linear independent set")
    else:
        n = len(basis)
        M = matrix([basis[i].list() for i in range(n)])
        for i in range(n):
            for j in range(i+1, n):
                if matrix(M.rows() + [vector(lie_bracket(basis[i], basis[j]).list())]).rank() > n:
                    return False
        return True

## La grassmanniana $G_2^+(\mathbb R^5) = \operatorname{SO}(5) / (\operatorname{SO}(2) \times \operatorname{SO}(3))$

In [3]:
N, k = 5, 2
g_initial_basis = []
for i in range(N):
    for j in range(i+1,N):
        B = matrix(SR, N, N)
        B[i,j], B[j,i] = -1, 1
        g_initial_basis.append(B)

print(len(g_initial_basis))
is_lie_algebra(g_initial_basis)

10


True

In [4]:
g_initial_basis

[
[ 0 -1  0  0  0]  [ 0  0 -1  0  0]  [ 0  0  0 -1  0]  [ 0  0  0  0 -1]
[ 1  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 1  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 1  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0], [ 0  0  0  0  0], [ 0  0  0  0  0], [ 1  0  0  0  0],

[ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0 -1  0  0]  [ 0  0  0 -1  0]  [ 0  0  0  0 -1]  [ 0  0  0  0  0]
[ 0  1  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0 -1  0]
[ 0  0  0  0  0]  [ 0  1  0  0  0]  [ 0  0  0  0  0]  [ 0  0  1  0  0]
[ 0  0  0  0  0], [ 0  0  0  0  0], [ 0  1  0  0  0], [ 0  0  0  0  0],

[ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0 -1]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0 -1]
[ 0  0  1  0  0], [ 0  0  0  1  0]
]

In [5]:
h_initial_basis = []
for i in range(k):
    for j in range(i+1,k):
        B = matrix(SR, k, k)
        B[i,j], B[j,i] = -1, 1
        h_initial_basis.append(block_diagonal_matrix([B, matrix(SR, N-k, N-k)]))

for i in range(N-k):
    for j in range(i+1,N-k):
        B = matrix(SR, N-k, N-k)
        B[i,j], B[j,i] = -1, 1
        h_initial_basis.append(block_diagonal_matrix([matrix(SR, k, k), B]))

h_initial_basis

[
[ 0 -1| 0  0  0]  [ 0  0| 0  0  0]  [ 0  0| 0  0  0]  [ 0  0| 0  0  0]
[ 1  0| 0  0  0]  [ 0  0| 0  0  0]  [ 0  0| 0  0  0]  [ 0  0| 0  0  0]
[-----+--------]  [-----+--------]  [-----+--------]  [-----+--------]
[ 0  0| 0  0  0]  [ 0  0| 0 -1  0]  [ 0  0| 0  0 -1]  [ 0  0| 0  0  0]
[ 0  0| 0  0  0]  [ 0  0| 1  0  0]  [ 0  0| 0  0  0]  [ 0  0| 0  0 -1]
[ 0  0| 0  0  0], [ 0  0| 0  0  0], [ 0  0| 1  0  0], [ 0  0| 0  1  0]
]

In [6]:
def is_subalgebra(h_basis, g_basis):
    if is_lie_algebra(h_basis):
        for X in h_basis:
            if is_li(g_basis + [X]):
                return False
        return True

In [7]:
is_subalgebra(h_initial_basis, g_initial_basis)

True

In [8]:
def gram_schdmidt(basis, metric):
    on_basis = []
    for i, X in enumerate(basis):
        Y = X
        for j in range(i):
            Y -= metric(X, on_basis[j]) * on_basis[j]
        on_basis.append(1/sqrt(metric(Y, Y)) * Y)
    return on_basis

In [9]:
g_adapted_basis = h_initial_basis.copy()
for X in g_initial_basis:
    if is_li(g_adapted_basis + [X]):
        g_adapted_basis.append(X)

g_on_basis = gram_schdmidt(g_adapted_basis, biinv_metric)
h_on_basis = g_on_basis[:len(h_initial_basis)]

In [10]:
g_on_basis == g_initial_basis

False

In [11]:
g_on_basis

[
[           0 -1/2*sqrt(2)            0            0            0]
[ 1/2*sqrt(2)            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0            0],

[           0            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0 -1/2*sqrt(2)            0]
[           0            0  1/2*sqrt(2)            0            0]
[           0            0            0            0            0],

[           0            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0 -1/2*sqrt(2)]
[           0            0            0            0            0]
[           0            0  1/2*sqrt(2)            0            0],

[           0            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0 -1/2*sqrt(2)]
[           0            0            0  1/2*sqrt(2)            0],

[           0            0 -1/2*sqrt(2)            0            0]
[           0            0            0            0            0]
[ 1/2*sqrt(2)            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0            0],

[           0            0            0 -1/2*sqrt(2)            0]
[           0            0            0            0            0]
[           0            0            0            0            0]
[ 1/2*sqrt(2)            0            0            0            0]
[           0            0            0            0            0],

[           0            0            0            0 -1/2*sqrt(2)]
[           0            0            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0            0]
[ 1/2*sqrt(2)            0            0            0            0],

[           0            0            0            0            0]
[           0            0 -1/2*sqrt(2)            0            0]
[           0  1/2*sqrt(2)            0            0            0]
[           0            0            0            0            0]
[           0            0            0            0            0],

[           0            0            0            0            0]
[           0            0            0 -1/2*sqrt(2)            0]
[           0            0            0            0            0]
[           0  1/2*sqrt(2)            0            0            0]
[           0            0            0            0            0],

[           0            0            0            0            0]
[           0            0            0            0 -1/2*sqrt(2)]
[           0            0            0            0            0]
[           0            0            0            0            0]
[           0  1/2*sqrt(2)            0            0            0]
]

In [12]:
m_initial_basis = g_on_basis[len(h_initial_basis):]
n = len(m_initial_basis)

In [13]:
def proj_h(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in m_initial_basis])

def proj_m(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in h_on_basis])

In [14]:
# Test con metrica normal homogenea
metric_m = biinv_metric

In [15]:
m_on_basis = gram_schdmidt(m_initial_basis, metric_m)
print(matrix(n, n, lambda i, j: metric_m(m_initial_basis[i], m_initial_basis[j])))
print("")
print(matrix(n, n, lambda i, j: metric_m(m_on_basis[i], m_on_basis[j])))

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]


**Chequeamos la Ad-invariancia.** Por simplicidad trabajaremos al nivel del  álgebra de Lie verificando que $\operatorname{ad}_X |_{\mathfrak m}: \mathfrak m \to \mathfrak m$ sea antisimétrica para cada $X \in \mathfrak h$. Si denotamos por $X_1, \ldots, X_n$ la base ortonormal de $\mathfrak m$, entonces la matriz de $\operatorname{ad}_X$ en esta base esta dada por
$$(a_{ij}) = ( \langle [X, X_j], X_i\rangle)$$

In [16]:
def ad_matrix_m(X):
    return matrix(n, n, lambda i, j: metric_m(lie_bracket(X, m_on_basis[j]), m_on_basis[i]))

In [17]:
for X in h_on_basis:
    print(ad_matrix_m(X).is_skew_symmetric())

True
True
True
True


**Conexión de Levi-Civita:** para todos $X, Y \in \mathfrak m$, $$(\nabla_{X^*} Y^*)_o = - \frac12 [X, Y]_{\mathfrak m} + U(X, Y)$$ en donde $U: \mathfrak m \times \mathfrak m \to \mathfrak m$ está determinado implícitamente por la fórmula $$2 \langle U(X, Y), Z\rangle = \langle [Z, X]_{\mathfrak m}, Y \rangle + \langle X, [Z, Y]_{\mathfrak m}\rangle$$ para todo $Z \in \mathfrak m$


In [18]:
def U_tensor(X, Y):
    return (1/2) * sum([(metric_m(proj_m(lie_bracket(Z,X)),Y) + metric_m(X, proj_m(lie_bracket(Z, Y)))) * Z for Z in m_on_basis])

def levi_civita(X, Y):
    """
    Estamos identificando X, Y con los campos de Killing asociados!!!
    """
    return -(1/2) * proj_m(lie_bracket(X, Y)) + U_tensor(X, Y)

In [19]:
# Chequeamos si la metrica es naturalmente reductiva
# long time
%time set([U_tensor(X, Y) == 0 for X in m_on_basis for Y in m_on_basis])

CPU times: user 407 ms, sys: 2.95 ms, total: 410 ms
Wall time: 410 ms


{True}

**Curvatura seccional:** para todos $X, Y \in \mathfrak m$
\begin{align*}
\langle R(X, Y) X, Y\rangle  = & -\frac34 \langle [X, Y]_{\mathfrak m}, [X, Y]_{\mathfrak m}\rangle - \frac12 \langle [X, [X, Y]_{\mathfrak m} ]_{\mathfrak m}, Y\rangle \\
& -\frac12 \langle [Y, [Y, X]_{\mathfrak m} ]_{\mathfrak m}, X\rangle + \langle U(X, Y), U(X, Y)\rangle \\
& - \langle U(X, X), U(Y, Y) \rangle + \langle Y, [ [X, Y]_{\mathfrak h}, X]_{\mathfrak m}\rangle
\end{align*}


In [20]:
def sec_curv(X, Y):
    sum1 = -(3/4) * metric_m(proj_m(lie_bracket(X, Y)), proj_m(lie_bracket(X, Y)))
    sum2 = -(1/2) * metric_m(proj_m(lie_bracket(X, proj_m(lie_bracket(X, Y)))), Y)
    sum3 = -(1/2) * metric_m(proj_m(lie_bracket(Y, proj_m(lie_bracket(Y, X)))), X)
    sum4 = metric_m(U_tensor(X, Y), U_tensor(X, Y))
    sum5 = -metric_m(U_tensor(X, X), U_tensor(Y, Y))
    sum6 = metric_m(Y, proj_m(lie_bracket(proj_h(lie_bracket(X, Y)), X)))
    return sum1 + sum2 + sum3 + sum4 + sum5 + sum6

In [21]:
# long time
for i in range(n):
    for j in range(i+1,n):
        print(sec_curv(m_on_basis[i], m_on_basis[j]))
print("")
print("Fin.")

1/2
1/2
1/2
0
0
1/2
0
1/2
0
0
0
1/2
1/2
1/2
1/2

Fin.


**Curvatura de Ricci:** Fijemos una base ortonormal $X_1, \ldots, X_n$ de $\mathfrak m$. Entonces para todo $X \in \mathfrak m$ se tiene
\begin{align*}
\operatorname{Ric}(X, X) = & \sum_i \langle R(X, X_i) X, X_i\rangle \\
= & -\frac12 \sum_i \langle [X, X_i]_{\mathfrak m}, [X, X_i]_{\mathfrak m} \rangle - \frac12 \sum_i \langle [X, [X, X_i]_{\mathfrak m} ]_{\mathfrak m}, X_i\rangle \\
& -\sum_i \langle [ X, [X, X_i]_{\mathfrak h} ]_{\mathfrak m}, X_i\rangle + \frac14 \sum_{i, j} \langle [X_i, X_j]_{\mathfrak m}, X\rangle^2 \\
&- \langle [Z, X]_{\mathfrak m}, X \rangle,
\end{align*}
en donde $Z = \sum_i U(X_i, X_i)$ (el cual puede calcularse implícitamente por $\langle Z, X\rangle = \operatorname{tr} \operatorname {ad}_X$)


In [22]:
# Implementacion menos eficiente
#def ricci_quadratic_form(X):
#    return sum([sec_curv(X, Y) for Y in m_on_basis])

In [23]:
sum_Uii = sum([U_tensor(X, X) for X in m_on_basis])
%time XiXj = {(i,j): proj_m(lie_bracket(m_on_basis[i], m_on_basis[j])).canonicalize_radical() for i in range(n) for j in range(n)}

CPU times: user 1.62 s, sys: 36.3 ms, total: 1.66 s
Wall time: 1.22 s


In [24]:
def ricci_quadratic_form_1(X):
    XXi_m = [proj_m(lie_bracket(X, Y)) for Y in m_on_basis]
    XXXi_mm = [proj_m(lie_bracket(X, XXi_m[i])) for i in range(n)]
    XXXi_hm = [proj_m(lie_bracket(X,proj_h(lie_bracket(X, Y)))) for Y in m_on_basis]
    sum1 = -(1/2) * sum([metric_m(Y, Y) for Y in XXi_m])
    sum2 = -(1/2) * sum([metric_m(XXXi_mm[i], m_on_basis[i]) for i in range(n)])
    sum3 = -sum([metric_m(XXXi_hm[i], m_on_basis[i]) for i in range(n)])
    sum4 = (1/4) * sum([metric_m(XiXj[(i,j)], X)^2 for i in range(n) for j in range(n)])
    sum5 = -metric_m(proj_m(lie_bracket(sum_Uii, X)), X)
    return sum1 + sum2 + sum3 + sum4 + sum5

In [25]:
%time ricci_quadratic_form_1(m_on_basis[0])

CPU times: user 50.9 ms, sys: 1 μs, total: 50.9 ms
Wall time: 53.7 ms


3/2

In [26]:
#long time
for X in m_on_basis:
    print(ricci_quadratic_form_1(X))

3/2
3/2
3/2
3/2
3/2
3/2


Ahora definimos el tensor de Ricci, empezando por su forma cuadrática y luego polarizando

In [27]:
#def ricci_tensor(X, Y):
#    return (1/2) * (ricci_quadratic_form(X + Y) - ricci_quadratic_form(X) - ricci_quadratic_form(Y))

In [28]:
def ricci_tensor_1(X, Y):
    return (1/2) * (ricci_quadratic_form_1(X + Y) - ricci_quadratic_form_1(X) - ricci_quadratic_form_1(Y))

In [29]:
%time ricci_tensor_1(m_on_basis[0], m_on_basis[1])

CPU times: user 98.8 ms, sys: 0 ns, total: 98.8 ms
Wall time: 98.6 ms


0

In [30]:
Ric = matrix(SR,n,n)

# very long time
for i in range(n):
    for j in range(i,n):
        Ric[i,j] = Ric[j, i] = ricci_tensor_1(m_on_basis[i], m_on_basis[j]).canonicalize_radical()
        print('lugares ('+ str(i) + ',' +str(j) + ') y (' + str(j) + ',' + str(i) + ') completos...')
#        print(Ric)
        print(".............................")

lugares (0,0) y (0,0) completos...
.............................
lugares (0,1) y (1,0) completos...
.............................
lugares (0,2) y (2,0) completos...
.............................
lugares (0,3) y (3,0) completos...
.............................
lugares (0,4) y (4,0) completos...
.............................
lugares (0,5) y (5,0) completos...
.............................
lugares (1,1) y (1,1) completos...
.............................
lugares (1,2) y (2,1) completos...
.............................
lugares (1,3) y (3,1) completos...
.............................
lugares (1,4) y (4,1) completos...
.............................
lugares (1,5) y (5,1) completos...
.............................
lugares (2,2) y (2,2) completos...
.............................
lugares (2,3) y (3,2) completos...
.............................
lugares (2,4) y (4,2) completos...
.............................
lugares (2,5) y (5,2) completos...
.............................
lugares (3,3) y (3,3) com

In [31]:
Ric

[3/2   0   0   0   0   0]
[  0 3/2   0   0   0   0]
[  0   0 3/2   0   0   0]
[  0   0   0 3/2   0   0]
[  0   0   0   0 3/2   0]
[  0   0   0   0   0 3/2]

In [32]:
# long time, pero mejora mucho la eficiencia
%time Ric = matrix(n, n, lambda i, j: ricci_tensor_1(m_on_basis[i], m_on_basis[j])).canonicalize_radical()
Ric

CPU times: user 2.98 s, sys: 196 μs, total: 2.98 s
Wall time: 2.98 s


[3/2   0   0   0   0   0]
[  0 3/2   0   0   0   0]
[  0   0 3/2   0   0   0]
[  0   0   0 3/2   0   0]
[  0   0   0   0 3/2   0]
[  0   0   0   0   0 3/2]

In [33]:
# Cáculo explícito de la curvatura seccional

gen_vars_m = [var("x"+str(i)) for i in range(len(m_on_basis))]
gen_vars_m_1 = [var("y"+str(i)) for i in range(len(m_on_basis))]
m_gen = sum([gen_vars_m[i] * m_on_basis[i] for i in range(len(m_on_basis))])
m_gen_1 = sum([gen_vars_m_1[i] * m_on_basis[i] for i in range(len(m_on_basis))])
m_gen, m_gen_1

(
[              0               0 -1/2*sqrt(2)*x0 -1/2*sqrt(2)*x1 -1/2*sqrt(2)*x2]  [              0               0 -1/2*sqrt(2)*y0 -1/2*sqrt(2)*y1 -1/2*sqrt(2)*y2]
[              0               0 -1/2*sqrt(2)*x3 -1/2*sqrt(2)*x4 -1/2*sqrt(2)*x5]  [              0               0 -1/2*sqrt(2)*y3 -1/2*sqrt(2)*y4 -1/2*sqrt(2)*y5]
[ 1/2*sqrt(2)*x0  1/2*sqrt(2)*x3               0               0               0]  [ 1/2*sqrt(2)*y0  1/2*sqrt(2)*y3               0               0               0]
[ 1/2*sqrt(2)*x1  1/2*sqrt(2)*x4               0               0               0]  [ 1/2*sqrt(2)*y1  1/2*sqrt(2)*y4               0               0               0]
[ 1/2*sqrt(2)*x2  1/2*sqrt(2)*x5               0               0               0], [ 1/2*sqrt(2)*y2  1/2*sqrt(2)*y5               0               0               0]
)

In [34]:
sec_curv(m_gen, m_gen_1).factor()

1/2*x1^2*y0^2 + 1/2*x2^2*y0^2 + 1/2*x3^2*y0^2 - x0*x1*y0*y1 + x3*x4*y0*y1 + 1/2*x0^2*y1^2 + 1/2*x2^2*y1^2 + 1/2*x4^2*y1^2 - x0*x2*y0*y2 + x3*x5*y0*y2 - x1*x2*y1*y2 + x4*x5*y1*y2 + 1/2*x0^2*y2^2 + 1/2*x1^2*y2^2 + 1/2*x5^2*y2^2 - x0*x3*y0*y3 + x1*x4*y0*y3 + x2*x5*y0*y3 - 2*x0*x4*y1*y3 - 2*x0*x5*y2*y3 + 1/2*x0^2*y3^2 + 1/2*x4^2*y3^2 + 1/2*x5^2*y3^2 - 2*x1*x3*y0*y4 + x0*x3*y1*y4 - x1*x4*y1*y4 + x2*x5*y1*y4 - 2*x1*x5*y2*y4 + x0*x1*y3*y4 - x3*x4*y3*y4 + 1/2*x1^2*y4^2 + 1/2*x3^2*y4^2 + 1/2*x5^2*y4^2 - 2*x2*x3*y0*y5 - 2*x2*x4*y1*y5 + x0*x3*y2*y5 + x1*x4*y2*y5 - x2*x5*y2*y5 + x0*x2*y3*y5 - x3*x5*y3*y5 + x1*x2*y4*y5 - x4*x5*y4*y5 + 1/2*x2^2*y5^2 + 1/2*x3^2*y5^2 + 1/2*x4^2*y5^2

In [35]:
(sec_curv(m_gen, m_gen_1) / (metric_m(m_gen, m_gen) * metric_m(m_gen_1, m_gen_1) - metric_m(m_gen, m_gen_1)^2)).factor()

1/2*(x1^2*y0^2 + x2^2*y0^2 + x3^2*y0^2 - 2*x0*x1*y0*y1 + 2*x3*x4*y0*y1 + x0^2*y1^2 + x2^2*y1^2 + x4^2*y1^2 - 2*x0*x2*y0*y2 + 2*x3*x5*y0*y2 - 2*x1*x2*y1*y2 + 2*x4*x5*y1*y2 + x0^2*y2^2 + x1^2*y2^2 + x5^2*y2^2 - 2*x0*x3*y0*y3 + 2*x1*x4*y0*y3 + 2*x2*x5*y0*y3 - 4*x0*x4*y1*y3 - 4*x0*x5*y2*y3 + x0^2*y3^2 + x4^2*y3^2 + x5^2*y3^2 - 4*x1*x3*y0*y4 + 2*x0*x3*y1*y4 - 2*x1*x4*y1*y4 + 2*x2*x5*y1*y4 - 4*x1*x5*y2*y4 + 2*x0*x1*y3*y4 - 2*x3*x4*y3*y4 + x1^2*y4^2 + x3^2*y4^2 + x5^2*y4^2 - 4*x2*x3*y0*y5 - 4*x2*x4*y1*y5 + 2*x0*x3*y2*y5 + 2*x1*x4*y2*y5 - 2*x2*x5*y2*y5 + 2*x0*x2*y3*y5 - 2*x3*x5*y3*y5 + 2*x1*x2*y4*y5 - 2*x4*x5*y4*y5 + x2^2*y5^2 + x3^2*y5^2 + x4^2*y5^2)/(x1^2*y0^2 + x2^2*y0^2 + x3^2*y0^2 + x4^2*y0^2 + x5^2*y0^2 - 2*x0*x1*y0*y1 + x0^2*y1^2 + x2^2*y1^2 + x3^2*y1^2 + x4^2*y1^2 + x5^2*y1^2 - 2*x0*x2*y0*y2 - 2*x1*x2*y1*y2 + x0^2*y2^2 + x1^2*y2^2 + x3^2*y2^2 + x4^2*y2^2 + x5^2*y2^2 - 2*x0*x3*y0*y3 - 2*x1*x3*y1*y3 - 2*x2*x3*y2*y3 + x0^2*y3^2 + x1^2*y3^2 + x2^2*y3^2 + x4^2*y3^2 + x5^2*y3^2 - 2*x0*x4*y0*y4 - 2*x1*x4*y1*y4 - 2*x2*x4*y2*y4 - 2*x3*x4*y3*y4 + x0^2*y4^2 + x1^2*y4^2 + x2^2*y4^2 + x3^2*y4^2 + x5^2*y4^2 - 2*x0*x5*y0*y5 - 2*x1*x5*y1*y5 - 2*x2*x5*y2*y5 - 2*x3*x5*y3*y5 - 2*x4*x5*y4*y5 + x0^2*y5^2 + x1^2*y5^2 + x2^2*y5^2 + x3^2*y5^2 + x4^2*y5^2)